# 2022 빅콘테스트_spec 데이터 전처리
## 우 걱정마세요

##1. 라이브러리 및 패키지 불러오기

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
user_spec = pd.read_csv('/content/gdrive/MyDrive/BigContest_data/user_spec.csv')
user_spec.head()

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1.0,27000000.0
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5.0,15000000.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,자가,2000000.0,생활비,0.0,NaN,7.0,344000000.0
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,16000000.0


In [ ]:
loan_result = pd.read_csv('/content/gdrive/MyDrive/BigContest_data/loan_result.csv')
loan_result

##2. 결측값이 9개 이상인 row 제거

In [ ]:
respec = user_spec
drop_application_id = list(set(respec['application_id'])-set(loan_result['application_id']))

In [ ]:
respec.drop(respec[respec['application_id'].isin(drop_application_id)].index,inplace=True)

##3. yearly_income 컬럼

In [ ]:
respec[respec['yearly_income'].isnull()]

In [ ]:
respec.loc[respec['yearly_income'].isnull(), 'yearly_income'] = 0

## 4. purpose 컬럼

In [ ]:
respec['purpose'].unique()

In [ ]:
respec['purpose'] = respec['purpose'].replace("기타", 'ETC').replace("대환대출", 'SWITCHLOAN').replace("생활비", 'LIVING').replace("사업자금", "BUSINESS").replace("주택구입", 'BUYHOUSE').replace("전월세보증금", 'HOUSEDEPOSIT').replace("투자", 'INVEST').replace("자동차구입", 'BUYCAR')


In [ ]:
respec.groupby(['purpose']).mean()

##5. birth_year 컬럼
* birth_year을 나이로 바꿔주었음.
* 나이를 10단위로 바꿔주었고, 70이상의 값은 70으로 처리
* yearly_income을 기준으로 null값을 채움

In [ ]:
# 유저 성별이 다른 신청서에는 있지만 결측치가 있는 유저들 처리
gender_null = set(respec[(respec['user_id'].isin(set(respec[respec['gender'].isnull()]['user_id'])))&(respec['gender'].notnull())]['user_id'])

In [ ]:
for i in gender_null:
    respec.loc[(respec['user_id']==i)&(respec['birth_year'].isnull()),'birth_year'] = respec[respec['user_id'] == i]['birth_year'].dropna().unique().item()
    respec.loc[(respec['user_id']==i)&(respec['gender'].isnull()),'gender'] = respec[respec['user_id'] == i]['gender'].dropna().unique().item()

In [ ]:
plt.figure(figsize=(30,8))
sns.distplot(respec['birth_year'])
plt.show()

In [ ]:
respec['age'] = 2023 - respec['birth_year']

In [ ]:
plt.figure(figsize=(30,8))
sns.distplot(respec['age'])
plt.show()

In [ ]:
respec.loc[respec['age'] < 20, 'reage'] = 10
respec.loc[(respec['age'] >= 20) & (respec['age'] <30), 'reage'] = 20
respec.loc[(respec['age'] >= 30) & (respec['age'] <40), 'reage'] = 30
respec.loc[(respec['age'] >= 40) & (respec['age'] <50), 'reage'] = 40
respec.loc[(respec['age'] >= 50) & (respec['age'] <60), 'reage'] = 50
respec.loc[(respec['age'] >= 60) & (respec['age'] <70), 'reage'] = 60
respec.loc[respec['age'] >= 70, 'reage'] = 70

In [ ]:
respec_group = respec.groupby(['reage']).mean()
respec_group = respec_group.reset_index()

In [ ]:
respec_group

In [ ]:
respec.loc[(respec['reage'].isnull()) & (respec['yearly_income'] < respec_group[respec_group['reage'] == 20]['yearly_income'].item()), 'reage'] = 10
respec.loc[(respec['reage'].isnull()) & (respec['yearly_income'] >= respec_group[respec_group['reage'] == 20]['yearly_income'].item()) & (respec['yearly_income'] < respec_group[respec_group['reage'] == 70]['yearly_income'].item()), 'reage'] = 20
respec.loc[(respec['reage'].isnull()) & (respec['yearly_income'] >= respec_group[respec_group['reage'] == 70]['yearly_income'].item()) & (respec['yearly_income'] < respec_group[respec_group['reage'] == 30]['yearly_income'].item()), 'reage'] = 70
respec.loc[(respec['reage'].isnull()) & (respec['yearly_income'] >= respec_group[respec_group['reage'] == 30]['yearly_income'].item()) & (respec['yearly_income'] < respec_group[respec_group['reage'] == 60]['yearly_income'].item()), 'reage'] = 30
respec.loc[(respec['reage'].isnull()) & (respec['yearly_income'] >= respec_group[respec_group['reage'] == 60]['yearly_income'].item()) & (respec['yearly_income'] < respec_group[respec_group['reage'] == 40]['yearly_income'].item()	), 'reage'] = 60
respec.loc[(respec['reage'].isnull()) & (respec['yearly_income'] >= respec_group[respec_group['reage'] == 40]['yearly_income'].item()) & (respec['yearly_income'] < respec_group[respec_group['reage'] == 50]['yearly_income'].item()), 'reage'] = 40
respec.loc[(respec['reage'].isnull()) & (respec['yearly_income'] >= respec_group[respec_group['reage'] == 50]['yearly_income'].item()) , 'reage'] = 50

In [ ]:
respec = respec.drop(['birth_year', 'age'], axis = 1)

## 6. gender 컬럼

In [ ]:
respec['income_type'].unique()

In [ ]:
sample1 = respec[(respec['gender'].notnull()) & (respec['income_type'] == 'PRIVATEBUSINESS')]['gender'].value_counts()

per_0 = sample1[0] / (sample1[1] + sample1[0])
per_0 = (per_0*100).round(0)

In [ ]:
# income_type == 1
rnd_num = np.random.uniform(0, 1, 1)

if rnd_num > per_0:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'PRIVATEBUSINESS'), 'gender'] = 1
else:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'PRIVATEBUSINESS'), 'gender'] = 0

In [ ]:
sample1 = respec[(respec['gender'].notnull()) & (respec['income_type'] == 'FREELANCER')]['gender'].value_counts()

per_0 = sample1[0] / (sample1[1] + sample1[0])
per_0 = (per_0*100).round(0)

In [ ]:
# income_type == 2
rnd_num = np.random.uniform(0, 1, 1)

if rnd_num > per_0:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'FREELANCER'), 'gender'] = 1
else:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'FREELANCER'), 'gender'] = 0

In [ ]:
sample1 = respec[(respec['gender'].notnull()) & (respec['income_type'] == 'EARNEDINCOME')]['gender'].value_counts()

per_0 = sample1[0] / (sample1[1] + sample1[0])
per_0 = (per_0*100).round(0)

In [ ]:
# income_type == 3
rnd_num = np.random.uniform(0, 1, 1)

if rnd_num > per_0:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'EARNEDINCOME'), 'gender'] = 1
else:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'EARNEDINCOME'), 'gender'] = 0

In [ ]:
sample1 = respec[(respec['gender'].notnull()) & (respec['income_type'] == 'OTHERINCOME')]['gender'].value_counts()

per_0 = sample1[0] / (sample1[1] + sample1[0])
per_0 = (per_0*100).round(0)

In [ ]:
# income_type == 4
rnd_num = np.random.uniform(0, 1, 1)

if rnd_num > per_0:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'OTHERINCOME'), 'gender'] = 1
else:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'OTHERINCOME'), 'gender'] = 0

In [ ]:
sample1 = respec[(respec['gender'].notnull()) & (respec['income_type'] == 'EARNEDINCOME2')]['gender'].value_counts()

per_0 = sample1[0] / (sample1[1] + sample1[0])
per_0 = (per_0*100).round(0)

In [ ]:
# income_type == 5
rnd_num = np.random.uniform(0, 1, 1)

if rnd_num > per_0:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'EARNEDINCOME2'), 'gender'] = 1
else:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'EARNEDINCOME2'), 'gender'] = 0

In [ ]:
sample1 = respec[(respec['gender'].notnull()) & (respec['income_type'] == 'PRACTITIONER')]['gender'].value_counts()

per_0 = sample1[0] / (sample1[1] + sample1[0])
per_0 = (per_0*100).round(0)

In [ ]:
# income_type == 6
rnd_num = np.random.uniform(0, 1, 1)

if rnd_num > per_0:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'PRACTITIONER'), 'gender'] = 1
else:
  respec.loc[(respec['gender'].isnull()) & (respec['income_type'] == 'PRACTITIONER'), 'gender'] = 0

In [ ]:
respec['gender'] = respec['gender'].astype(str)

## 7. company_enter_month 컬럼

In [ ]:
respec.loc[(respec['company_enter_month'] < 10000000),'company_enter_month'] = respec['company_enter_month'] / 100

In [ ]:
respec.loc[(respec['company_enter_month'] >= 10000000),'company_enter_month'] = respec['company_enter_month'] / 10000

In [ ]:
respec['company_enter_month'] = respec['company_enter_month'].round()

In [ ]:
respec.groupby(['reage'])['company_enter_month'].mean()

In [ ]:
respec.loc[(respec['company_enter_month'].isnull()) & (respec['reage'] <= 10),'company_enter_month'] = 2020
respec.loc[(respec['company_enter_month'].isnull()) & (respec['reage'] > 10) & (respec['reage'] <= 20),'company_enter_month'] = 2021
respec.loc[(respec['company_enter_month'].isnull()) & (respec['reage'] > 20) & (respec['reage'] <= 30),'company_enter_month'] = 2020
respec.loc[(respec['company_enter_month'].isnull()) & (respec['reage'] > 30) & (respec['reage'] <= 40),'company_enter_month'] = 2017
respec.loc[(respec['company_enter_month'].isnull()) & (respec['reage'] > 40) & (respec['reage'] <= 50),'company_enter_month'] = 2016
respec.loc[(respec['company_enter_month'].isnull()) & (respec['reage'] > 50) & (respec['reage'] <= 60),'company_enter_month'] = 2017
respec.loc[(respec['company_enter_month'].isnull()) & (respec['reage'] > 60),'company_enter_month'] = 2018

In [ ]:
respec['career'] = 2022 - respec['company_enter_month'] + 1

In [ ]:
respec = respec.drop(['company_enter_month'], axis = 1)

## 7. existing_loan_cnt

In [ ]:
respec.loc[respec['existing_loan_cnt'].isnull(), 'existing_loan_cnt'] = 0

## 8. existing_loan_amt

In [ ]:
respec.loc[(respec['existing_loan_amt'].isnull()) & (respec['existing_loan_cnt']==0), 'existing_loan_amt'] = 0

In [ ]:
print(respec['existing_loan_amt'].describe())

In [ ]:
respec.loc[(respec['existing_loan_amt'].isnull()), 'existing_loan_amt'] = (7.700000e+07 + 1.000000e+07) /2

## 9. personal_rehabilitation_yn

In [ ]:
pry_400 =respec[respec['credit_score']<=400]
pry_400.head()

In [ ]:
respec['personal_rehabilitation_yn'] = pry_400['personal_rehabilitation_yn'].fillna(1)

In [ ]:
respec['personal_rehabilitation_yn'].isnull().sum()

In [ ]:
df=respec[respec['personal_rehabilitation_yn'].isnull()]
df_sample = df.sample(n=569011, random_state=100)
df_sample

In [ ]:
respec['personal_rehabilitation_yn'] = df_sample['personal_rehabilitation_yn'].fillna(0)

In [ ]:
respec['personal_rehabilitation_yn'].isnull().sum()

In [ ]:
respec['personal_rehabilitation_yn'] = respec['personal_rehabilitation_yn'].fillna(1)

In [ ]:
respec['personal_rehabilitation_yn'].isnull().sum()

## 10. personal_rehabilitation_complete_yn

In [ ]:
pr_0 = respec['personal_rehabilitation_yn'] == 0
prc_0 = respec['personal_rehabilitation_complete_yn'] == 0

subset = respec[pr_0 & prc_0]
subset

In [ ]:
subset['personal_rehabilitation_complete_yn'] =subset['personal_rehabilitation_complete_yn'].replace(0, 2)
subset

In [ ]:
pr_1 = respec['personal_rehabilitation_yn'] != 0
prc_1 = respec['personal_rehabilitation_complete_yn'] != 0

subset1 = respec[pr_1 | prc_1]

In [ ]:
prcy_600 =respec[respec['credit_score']>=600]
prcy_600.head()

In [ ]:
respec['personal_rehabilitation_complete_yn'] = prcy_600['personal_rehabilitation_complete_yn'].fillna(1)

In [ ]:
respec['personal_rehabilitation_complete_yn'].isnull().sum()

In [ ]:
respec['personal_rehabilitation_complete_yn'] = respec['personal_rehabilitation_complete_yn'].fillna(0)

In [ ]:
respec['personal_rehabilitation_complete_yn'].isnull().sum()

In [ ]:
respec.isnull().sum()

In [ ]:
respec.to_csv("respec.csv", index = False)